## 创建项目环境和基本依赖

In [ ]:
uv init deep-research-quickstart
cd deep-research-quickstart
mkdir src
uv add jinja2 firecrawl-py socksio
uv add langchain langchain-openai langchain_tavily
uv add langgraph langgraph-supervisor

## 一、集成模型

In [1]:
import os
from src.models.chat_model import chat_model_doubao

print(chat_model_doubao.invoke("你好，豆包").content)

你好呀！有什么问题我都可以帮你解答，尽管跟我说。 


In [2]:
import os
from src.models.chat_model import chat_model

print(chat_model.invoke("你好").content)

你好！有什么我可以帮助你的吗？


## 二、使用Meta prompt生成提示词
这段提示词（META_PROMPT）是一个元提示词生成器，它的作用是帮助用户创建高质量的系统提示词。

In [ ]:
from openai import OpenAI

client = OpenAI()

META_PROMPT = """
Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.
   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

[Concise instruction describing the task - this should be the first line in the prompt, no section header]

[Additional details as needed.]

[Optional sections with headings or bullet points for detailed steps.]

# Steps [optional]

[optional: a detailed breakdown of the steps necessary to accomplish the task]

# Output Format

[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]

# Examples [optional]

[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]
[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]

# Notes [optional]

[optional: edge cases, details, and an area to call or repeat out specific important considerations]
""".strip()

def generate_prompt(task_or_prompt: str):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": META_PROMPT,
            },
            {
                "role": "user",
                "content": "Task, Goal, or Current Prompt:\n" + task_or_prompt,
            },
        ],
    )

    return completion.choices[0].message.content


# 假设 generate_prompt 已经定义好并可用

# 1. 定义你的任务描述
task = "我要实现一个RAG（检索增强生成）系统，要求根据用户查询从知识库检索相关信息，并基于检索内容生成高质量的回答。"

# 2. 调用 generate_prompt 生成系统提示词
prompt = generate_prompt(task)

# 3. 格式化输出（比如用 print，或 Jupyter 的 display）
print("=== RAG 系统系统提示词 ===")
print(prompt)

In [ ]:
提示词功能

In [7]:

from openai import OpenAI

client = OpenAI()

META_PROMPT = """
Given a current prompt and a change description, produce a detailed system prompt to guide a language model in completing the task effectively.

Your final output will be the full corrected prompt verbatim. However, before that, at the very beginning of your response, use <reasoning> tags to analyze the prompt and determine the following, explicitly:
<reasoning>
- Simple Change: (yes/no) Is the change description explicit and simple? (If so, skip the rest of these questions.)
- Reasoning: (yes/no) Does the current prompt use reasoning, analysis, or chain of thought? 
    - Identify: (max 10 words) if so, which section(s) utilize reasoning?
    - Conclusion: (yes/no) is the chain of thought used to determine a conclusion?
    - Ordering: (before/after) is the chain of though located before or after 
- Structure: (yes/no) does the input prompt have a well defined structure
- Examples: (yes/no) does the input prompt have few-shot examples
    - Representative: (1-5) if present, how representative are the examples?
- Complexity: (1-5) how complex is the input prompt?
    - Task: (1-5) how complex is the implied task?
    - Necessity: ()
- Specificity: (1-5) how detailed and specific is the prompt? (not to be confused with length)
- Prioritization: (list) what 1-3 categories are the MOST important to address.
- Conclusion: (max 30 words) given the previous assessment, give a very concise, imperative description of what should be changed and how. this does not have to adhere strictly to only the categories listed
</reasoning>
    
# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.
   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

[Concise instruction describing the task - this should be the first line in the prompt, no section header]

[Additional details as needed.]

[Optional sections with headings or bullet points for detailed steps.]

# Steps [optional]

[optional: a detailed breakdown of the steps necessary to accomplish the task]

# Output Format

[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]

# Examples [optional]

[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]
[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]

# Notes [optional]

[optional: edge cases, details, and an area to call or repeat out specific important considerations]
[NOTE: you must start with a <reasoning> section. the immediate next token you produce should be <reasoning>]
""".strip()

def generate_prompt(task_or_prompt: str):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": META_PROMPT,
            },
            {
                "role": "user",
                "content": "Task, Goal, or Current Prompt:\n" + task_or_prompt,
            },
        ],
    )

    return completion.choices[0].message.content


# 1. 定义你的任务描述
task = "我要实现一个RAG（检索增强生成）系统，要求根据用户查询从知识库检索相关信息，并基于检索内容生成高质量的回答。"

# 2. 调用 generate_prompt 生成系统提示词
prompt = generate_prompt(task)

# 3. 格式化输出
print("=== RAG 系统系统提示词 ===")
print(prompt)

=== RAG 系统系统提示词 ===
<reasoning>
- Simple Change: yes
- Specificity: 2 how detailed and specific is the prompt? (not to be confused with length)
- Prioritization: Understand Task, Clarity, Output Format
- Conclusion: Clearly define the task and add guidance on the retrieval and generation process.
</reasoning>

Create a retrieval-augmented generation (RAG) system that retrieves relevant information from a knowledge base based on user queries and generates high-quality answers based on the retrieved content.

# Steps

- Accept a user query.
- Retrieve relevant information from the knowledge base using the query.
- Use the retrieved information to generate a high-quality answer.

# Output Format

- Answer should be in a concise paragraph format.

# Notes

- Ensure that the retrieved information is relevant and accurate.
- Focus on generating coherent and contextually appropriate answers.


In [12]:
from langchain.schema import SystemMessage, HumanMessage

import json
from src.models import chat_model

res = chat_model.invoke(
    [
        SystemMessage(
            content="""Provide answers to the user's questions based on the context they provide.

# Steps

1. Carefully read and understand the context provided by the user.
2. Analyze the question in relation to the given context.
3. Use logical reasoning to derive the answer based on the context.
4. If the context is insufficient to answer the question, politely ask for clarification or additional details.

# Output Format

- Provide a clear and concise answer in complete sentences.
- If reasoning is required, include the reasoning process before presenting the conclusion.

# Notes

- Ensure the response is directly relevant to the user's question and context.
- Avoid making assumptions beyond the provided context unless explicitly requested."""
        ),
        HumanMessage(
            content=json.dumps(
                [
                    {
                        "title": "欧国联决赛:C罗破门，带队夺冠，葡萄牙点球6-5击败西班牙!|c罗|门迪|尼科|葡萄牙队|豪尔赫·门德斯_网易订阅",
                        "url": "https://www.163.com/dy/article/K1JCRTCN05567SBP.html",
                        "content": "北京时间6月9日，欧国联决赛，葡萄牙2-2战平西班牙，点球大战总比分6-5取胜夺冠!这场比赛踢得非常精彩，西班牙的传控确实厉害，他们整体的技术和脚法是世界顶级的，西班牙队无论面对任何对手，都能在控球率方面占据优势，而葡萄牙同样很强，在推进到对方的禁区前沿时，葡萄牙有很多种",
                    },
                    {
                        "title": "欧国联决赛：葡萄牙5-3点球胜西班牙，C罗闪耀，金球奖归属引关注 - 知乎",
                        "url": "https://zhuanlan.zhihu.com/p/1915618913749795333",
                        "content": "欧国联 最后一轮踢完了。 葡萄牙在决赛5-3点球战胜西班牙，第二次拿冠军。法国打德国2-0赢了，拿了第三。c罗这场比赛进了球，成了国家队历史第一射手， 金球奖 基本没悬念了。 法国和德国那场球， 姆巴佩 表现很猛。 上半场他带球进禁区自己射门得分，下半场又给队友传球让对方空门进球。",
                    },
                    {
                        "title": "欧国联决赛葡萄牙点球5-3胜西班牙，C罗门德斯建功",
                        "url": "https://sports.sina.com.cn/g/2025-06-09/doc-inezmvxn4600761.shtml",
                        "content": "🆚欧国联a级决赛|葡萄牙2-2西班牙（点球5-3）🎯努诺·门德斯传射建功，巴黎四人随队夺冠🎞第21分钟，苏比门迪补射破门，西班牙1-0葡萄牙。🎞第",
                    },
                    {
                        "title": "欧国联：葡萄牙点球大战7-5西班牙第二次夺冠 C罗破门+伤退",
                        "url": "https://news.qq.com/rain/a/20250609A01I7S00",
                        "content": "欧国联-葡萄牙2-2点球5-3西班牙第2次欧国联夺冠!c罗破门扳平+伤退，莫拉塔失点. 北京时间6月9日凌晨，欧国联迎来最终的决赛，葡萄牙大战西班牙。",
                    },
                    {
                        "title": "40岁C罗收获生涯第36冠!葡萄牙点球7-5战胜西班牙，欧国联夺冠_澎湃号·媒体_澎湃新闻-The Paper",
                        "url": "https://www.thepaper.cn/newsDetail_forward_30952562",
                        "content": "40岁C罗收获生涯第36冠！葡萄牙点球7-5战胜西班牙，欧国联夺冠_澎湃号·媒体_澎湃新闻-The Paper Image 1: 澎湃Logo *   _要闻_ *   _深度_ *   _直播_ *   _视频_ *   _更多_ **下载客户端** 登录 无障碍) 40岁C罗收获生涯第36冠！葡萄牙点球7-5战胜西班牙，欧国联夺冠 Image 2 上游新闻 关注 重庆 来源：澎湃新闻·澎湃号·媒体 字号 北京时间6月9日，在欧国联决赛中，葡萄牙点球7-5战胜西班牙，队史第二次夺得欧国联冠军！40岁C罗连场破门，门德斯建功，祖比门迪、奥亚萨瓦尔破门。 Image 3Image 4 葡萄牙国家队官方发文庆祝夺冠： 我们又做到了！第二次捧起欧国联的奖杯，属于葡萄牙的荣耀，再次刻进这段绿茵史。不是奇迹，是坚持；不是偶然，是一代又一代的传承与信念。我们，是欧国联历史上夺冠最多的那支队伍！ Image 5 本场比赛是C罗国家队生涯至今参加的第4场决赛，此前分别是2004年欧洲杯决赛、2016年欧洲杯决赛（因伤离场）、2019年欧国联决赛，仅2004年欧洲杯未能夺冠。至此，40岁C罗职业生涯已夺36冠：国家队3冠，俱乐部33冠。 Image 6 赛后在接受采访时，C罗谈到了自己的伤势，他表示，热身时自己就感觉有些不适。 C罗这样谈道：“我在热身时就已经感觉到了，已经有一段时间了。但为了国家队，即使要断腿我也愿意。这是一个冠军，我必须上场，我尽了全力，坚持到了最后一刻，还打进了一球。” “我非常高兴。首先是为了这一代球员，他们值得一个冠军。为了我们的家人，我的家人都在这里……为葡萄牙赢得胜利是特别的。我有很多冠军头衔，但没有什么比为葡萄牙赢得胜利更美好的了。泪水和完成使命的感觉……这是美妙的。这是我们的国家。我们是一个小国，但有着远大的抱负。” “我曾在很多国家和俱乐部踢过球，但当人们提到葡萄牙时，那种感觉是特别的。作为这支球队的队长，我感到非常自豪，赢得冠军总是国家队的最高荣誉。未来我会考虑短期目标，我受了伤，而且伤势加重了……但我还是坚持了下来，因为为了国家队你必须全力以赴。” 关于主教练马丁内斯，C罗表示：“我为他感到非常高兴，他是一个西班牙人，但他为我们的国家付出了最大的努力。我们已经赢得了这个冠军，但这只是我们的动力，是我们渴望更多的开始。” Image 7 知名体育评论员詹俊赛后点评道： 逆转之夜！葡萄牙队两次落后两次扳平比分，最终在点球决战力克西班牙再次夺得欧国联冠军。C罗vs亚马尔，2003年18岁的C罗迎来在国家队的首秀，二十二年后40岁的C罗仍然能打进关键入球拿到个人第三项国际大赛的冠军。而亚马尔略显平淡因为碰到克星——努诺-门德斯，这位大巴黎的左后卫是个人心目中本场的MVP，“世界第一左后卫”的封号当之无愧。西班牙队的右路防守是软肋，明年世界杯他们还是需要卡瓦哈尔。 Image 8 本文为澎湃号作者或机构在澎湃新闻上传并发布，仅代表该作者或机构观点，不代表澎湃新闻的观点或立场，澎湃新闻仅提供信息发布平台。申请澎湃号请用电脑访问http://renzheng.thepaper.cn。 *   Image 9### C罗进球封王！这支葡萄牙让人想起了2022年的阿根廷  *   Image 10### C罗语出惊人：金球奖失去公信力  *   Image 11### 体坛联播｜C罗进球助葡萄牙夺冠，阿尔拉卡斯卫冕法网  *   Image 12_01:00_### 王大雷国脚生涯谢幕倒计时：希望给年轻球员做好榜样  *   Image 13### C罗进球封王！这支葡萄牙让人想起了2022年的阿根廷  *   Image 14### C罗语出惊人：金球奖失去公信力  *   Image 15### 体坛联播｜C罗进球助葡萄牙夺冠，阿尔拉卡斯卫冕法网  *   Image 16_01:00_### 王大雷国脚生涯谢幕倒计时：希望给年轻球员做好榜样  *   Image 17### C罗进球封王！这支葡萄牙让人想起了2022年的阿根廷  *   Image 18### C罗语出惊人：金球奖失去公信力  *   Image 19### 体坛联播｜C罗进球助葡萄牙夺冠，阿尔拉卡斯卫冕法网  *   Image 20_01:00_### 王大雷国脚生涯谢幕倒计时：希望给年轻球员做好榜样  Image 21 Image 22 Image 23 Image 24 Image 25 Image 26 *   报料邮箱: news@thepaper.cn Image 28 Image 29",
                    },
                ]
            )
        ),
        HumanMessage(content="欧足联决赛有什么看点？"),
    ]
)
print(res.content)

欧足联决赛有诸多看点，具体如下：
- **比赛过程精彩**：西班牙队的传控确实厉害，整体的技术和脚法是世界顶级的，无论面对任何对手，都能在控球率方面占据优势。而葡萄牙同样很强，在推进到对方的禁区前沿时，葡萄牙有很多种进攻方式。这场比赛双方常规时间2 - 2战平，点球大战更是紧张刺激，充分展现了比赛的激烈程度和不可预测性。
- **C 罗表现出色**：40岁的 C 罗连续两场破门，成为国家队历史第一射手，基本锁定金球奖。他在比赛中带伤坚持，为了国家队拼尽全力，这种精神令人敬佩。并且他作为葡萄牙队的队长，带领球队第二次捧起欧足联的奖杯，其领袖作用和赛场影响力是一大看点。
- **球员表现亮眼**：这场比赛中，马巴佩表现火热，上半场他带球进禁区自己射门得分，下半场又给队友传球让对方空门进球。另外，大巴塞罗那的左后卫努奥 - 门德斯是本场的 MVP，他被称为“世界第一左后卫”，实至名归。
- **球队荣誉意义**：对于葡萄牙国家队来说，这是他们队史第二次夺得欧足联冠军，这是属于葡萄牙的荣耀，体现了一代又一代的传承与信念。葡萄牙是欧足联历史上夺冠最多的那支队伍，此次夺冠再次书写辉煌队史。 


In [1]:
# 导入网页爬取工具
from src.tools.web_crawl import web_crawl
import os

# 检查 FIRECRAWL_API_KEY 环境变量
if not os.getenv("FIRECRAWL_API_KEY"):
    print("⚠️  警告: 未设置 FIRECRAWL_API_KEY 环境变量")
    print("请设置环境变量: export FIRECRAWL_API_KEY='your_api_key'")
else:
    print("✅ FIRECRAWL_API_KEY 已配置")

# 基本爬取示例
try:
    # 爬取一个示例网页
    url = "https://www.python.org"
    print(f"\n正在爬取: {url}")
    
    content = web_crawl.invoke(url)
    
    print(f"\n爬取成功! 内容预览:")
    print("=" * 50)
    # 只显示前500个字符
    print(content[:500] + "...")
    
except Exception as e:
    print(f"爬取失败: {e}")
    print("请确保已设置正确的 FIRECRAWL_API_KEY")


✅ FIRECRAWL_API_KEY 已配置

正在爬取: https://www.python.org

爬取成功! 内容预览:
**Notice:** While JavaScript is not essential for this website, your interaction with the content will be limited. Please turn JavaScript on for the full experience.

## Get Started

Whether you're new to programming or an experienced developer, it's easy to learn and use Python.

[Start with our Beginner’s Guide](https://www.python.org/about/gettingstarted/)

## Download

Python source code and installers are available for download for all versions!

Latest: [Python 3.13.5](https://www.python.o...


In [2]:
# 高级示例：爬取网页内容并用AI分析
from tools.models.chat_model import chat_model
from langchain.schema import SystemMessage, HumanMessage

def analyze_webpage(url, question):
    """爬取网页并用AI分析内容"""
    try:
        print(f"正在爬取: {url}")
        
        # 爬取网页内容
        content = web_crawl.invoke(url)
        
        if not content:
            return "爬取失败或内容为空"
            
        print(f"爬取成功，内容长度: {len(content)} 字符")
        
        # 使用AI分析内容
        response = chat_model.invoke([
            SystemMessage(content="""你是一个专业的内容分析师。请基于提供的网页内容回答用户的问题。
            要求：
            1. 仔细阅读网页内容
            2. 针对用户问题给出准确回答
            3. 如果内容不足以回答问题，请明确说明
            4. 用中文回答"""),
            HumanMessage(content=f"网页内容:\n{content[:3000]}\n\n用户问题: {question}")
        ])
        
        return response.content
        
    except Exception as e:
        return f"分析失败: {e}"

# 示例使用
if os.getenv("FIRECRAWL_API_KEY") and os.getenv("DOUBAO_API_KEY"):
    # 分析Python官网
    result = analyze_webpage(
        "https://www.python.org", 
        "这个网站主要介绍了什么？有哪些重要功能？"
    )
    print("\nAI分析结果:")
    print("=" * 50)
    print(result)
else:
    print("需要设置 FIRECRAWL_API_KEY 和 DOUBAO_API_KEY 环境变量才能运行此示例")


正在爬取: https://www.python.org
爬取成功，内容长度: 2397 字符

AI分析结果:
这个网站主要围绕Python编程语言展开介绍，涵盖了Python学习、下载、文档、工作机会、新闻、活动、成功案例、应用场景以及Python软件基金会等方面的内容。

重要功能如下：
1. **学习引导**：为编程新手和有经验的开发者提供Python学习途径，有初学者指南，链接为[Start with our Beginner’s Guide](https://www.python.org/about/gettingstarted/)。
2. **下载功能**：提供所有版本的Python源代码和安装程序下载，最新版本是Python 3.13.5，下载链接为[Python 3.13.5](https://www.python.org/downloads/release/python-3135/)。
3. **文档查阅**：提供Python标准库的文档、教程和指南，可通过[docs.python.org](https://docs.python.org/)在线查阅。
4. **工作机会**：设有社区运营的工作板块，为求职者和招聘者提供Python相关工作岗位的交流平台，链接为[jobs.python.org](https://jobs.python.org/)。
5. **资讯获取**：提供Python的最新新闻、即将举办的活动信息和成功案例，分别可通过[More News](https://blog.python.org/)、[More Events](https://www.python.org/events/calendars/)、[More Success Stories](https://www.python.org/success-stories/)查看。
6. **应用展示**：展示Python的应用场景，更多应用可查看[More Applications](https://www.python.org/about/apps)。
7. **基金会相关**：介绍Python软件基金会的使命，用户可了解更多信息、成为会员或进行捐赠，链接分别为[Learn more](https://www.python.org/psf/)、[Become a Mem

# 二、集成工具

In [ ]:
# 导入网页爬取工具
from src.tools.web_crawl import web_crawl
import os

# 检查 FIRECRAWL_API_KEY 环境变量
if not os.getenv("FIRECRAWL_API_KEY"):
    print("⚠️  警告: 未设置 FIRECRAWL_API_KEY 环境变量")
    print("请设置环境变量: export FIRECRAWL_API_KEY='your_api_key'")
else:
    print("✅ FIRECRAWL_API_KEY 已配置")

# 基本爬取示例
try:
    # 爬取一个示例网页
    url = "https://www.python.org"
    print(f"\n正在爬取: {url}")
    
    content = web_crawl.invoke(url)
    
    print(f"\n爬取成功! 内容预览:")
    print("=" * 50)
    # 只显示前500个字符
    print(content[:500] + "...")
    
except Exception as e:
    print(f"爬取失败: {e}")
    print("请确保已设置正确的 FIRECRAWL_API_KEY")
